# 🧠 Multimodal AI Model Scout — Twitter Top 5 (Last 7 Days)
**Narrow focus:** Recent multimodal model releases/benchmarks on par with Claude Sonnet or GPT-4o/5-class models  
**Geos:** San Francisco · Chicago · New York  
**Platforms:** Twitter (Yutori v2) + Tavily fallback  
**Output:** Top 5 highest-engagement tweets + top Tavily content ranked by viral score

In [ ]:
# ── Install ───────────────────────────────────────────────────────────────────
!pip install tavily-python requests -q

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# MULTIMODAL AI TWITTER SCOUT
# Finds top viral tweets + social content about new multimodal models
# that benchmark at Claude Sonnet / GPT-4o / GPT-5.2 level
# Geo-anchored: SF · Chicago · NYC tech communities
# ─────────────────────────────────────────────────────────────────────────────

import os, json, time, re
from getpass import getpass
from dataclasses import dataclass, field, asdict
from typing import List, Optional
from datetime import datetime, timezone
from collections import Counter

import requests
from tavily import TavilyClient


# ─────────────────────────────────────────────────────────────────────────────
# ★ CONFIG — Tune these to shift focus
# ─────────────────────────────────────────────────────────────────────────────

# Narrow topics: new multimodal models competing with Claude Sonnet / GPT-4o class
TOPICS = [
    "multimodal model benchmark Claude Sonnet GPT-4o",
    "new vision language model release 2025 2026",
    "Gemini Grok Llama multimodal release benchmark",
    "open source multimodal model GPT-5 Claude level",
    "multimodal AI demo event SF Chicago NYC",
]

# Geography anchors injected into Twitter queries
GEO_TERMS = ["San Francisco", "Chicago", "New York"]

# Recency window
RECENCY = "week"   # "day" | "week" | "month"

# How many Tavily results per query
MAX_RESULTS = 5

# Twitter: pull this many candidates per query, return top N
TWITTER_PULL = 50
TOP_N_TWEETS = 5    # ← final top tweets shown

# Tavily platforms to search
PLATFORMS = {
    "twitter":  True,
    "youtube":  True,   # demo/event videos
    "reddit":   True,
    "linkedin": False,
    "blogs":    False,
}


# ─────────────────────────────────────────────────────────────────────────────
# QUERY TEMPLATES — geo-injected for local tech community signal
# ─────────────────────────────────────────────────────────────────────────────

# Tavily query templates
QUERY_TEMPLATES = {
    "twitter": [
        "{topic} site:twitter.com OR site:x.com viral 2025 2026",
        "{topic} site:x.com release demo benchmark thread",
    ],
    "youtube": [
        "{topic} site:youtube.com demo release 2025 2026",
        "{topic} youtube multimodal benchmark review",
    ],
    "reddit": [
        "{topic} site:reddit.com r/MachineLearning OR r/LocalLLaMA OR r/artificial 2026",
        "{topic} reddit multimodal release hot 2025 2026",
    ],
}

# Twitter v2 queries — narrow + geo-anchored
TWITTER_V2_QUERIES = [
    # Core multimodal release signal
    '(multimodal model OR vision language model) (release OR launch OR benchmark) ("Claude Sonnet" OR "GPT-4o" OR "GPT-5") -is:retweet lang:en',
    # Tech event / demo in target cities
    '(multimodal AI OR vision model) ("San Francisco" OR "Chicago" OR "New York") (demo OR event OR keynote) -is:retweet lang:en',
    # Benchmark comparisons going viral
    '(new model OR multimodal) ("on par with" OR "beats" OR "matches" OR "rivals") (Claude OR GPT OR Sonnet) -is:retweet lang:en',
    # Open-source releases that got traction
    '(open source multimodal OR open-source VLM) (release OR launch OR "just dropped") 2025 OR 2026 -is:retweet lang:en',
    # Viral demo videos of multimodal models
    '(multimodal model) ("went viral" OR viral OR "blew up") (video OR demo OR thread) -is:retweet lang:en',
]


# ─────────────────────────────────────────────────────────────────────────────
# VIRAL SIGNALS
# ─────────────────────────────────────────────────────────────────────────────

VIRAL_SIGNAL_WORDS = [
    "went viral", "blew up", "breaking", "just released", "just dropped",
    "just announced", "everyone is talking", "thread", "repost",
    "comments", "likes", "views", "demo", "benchmark", "beats",
    "on par with", "rivals", "open source", "leaked", "exclusive",
    "multimodal", "vision", "release", "first look",
]

def detect_viral_signals(text):
    t = text.lower()
    return [s for s in VIRAL_SIGNAL_WORDS if s.lower() in t]

def detect_platform(url, fallback):
    u = url.lower()
    if "twitter.com" in u or "x.com" in u: return "twitter"
    if "youtube.com" in u or "youtu.be" in u: return "youtube"
    if "reddit.com" in u: return "reddit"
    if "linkedin.com" in u: return "linkedin"
    return fallback

def detect_content_type(url, title):
    u, t = url.lower(), title.lower()
    if "twitter.com" in u or "x.com" in u:
        return "thread" if "thread" in t else "post"
    if "youtube.com" in u or "youtu.be" in u: return "video"
    if "reddit.com" in u: return "thread"
    return "article"


# ─────────────────────────────────────────────────────────────────────────────
# DATA CLASSES
# ─────────────────────────────────────────────────────────────────────────────

@dataclass
class ScoutedPost:
    platform:        str
    content_type:    str
    title:           str
    url:             str
    snippet:         str
    full_content:    str
    relevance_score: float
    topic:           str
    query_used:      str
    published_date:  Optional[str]  = None
    viral_signals:   List[str]      = field(default_factory=list)

@dataclass
class ScoutOutput:
    agent:             str       = "multimodal_ai_scout"
    timestamp:         str       = field(default_factory=lambda: datetime.now(timezone.utc).isoformat())
    topics_scouted:    List[str] = field(default_factory=list)
    platforms_scouted: List[str] = field(default_factory=list)
    posts:             List[ScoutedPost] = field(default_factory=list)
    total_found:       int       = 0
    queries_run:       int       = 0


# ─────────────────────────────────────────────────────────────────────────────
# TAVILY SCOUT
# ─────────────────────────────────────────────────────────────────────────────

class TavilySocialScout:
    def __init__(self, api_key):
        self.client = TavilyClient(api_key=api_key)

    def _query(self, query, topic, platform, recency, max_results):
        days_map = {"day": 2, "week": 7, "month": 30}
        try:
            resp = self.client.search(
                query=query,
                search_depth="advanced",
                max_results=max_results,
                topic="news",
                days=days_map.get(recency, 7),
                include_answer=False,
                include_raw_content=False,
            )
        except Exception:
            try:
                resp = self.client.search(
                    query=query,
                    search_depth="advanced",
                    max_results=max_results,
                    include_answer=False,
                )
            except Exception as e:
                print(f"      ⚠️  Failed: {e}")
                return []
        posts = []
        for r in resp.get("results", []):
            url, title, content = r.get("url",""), r.get("title",""), r.get("content","")
            posts.append(ScoutedPost(
                platform        = detect_platform(url, platform),
                content_type    = detect_content_type(url, title),
                title           = title,
                url             = url,
                snippet         = content[:400],
                full_content    = content,
                relevance_score = round(r.get("score", 0.0), 4),
                topic           = topic,
                query_used      = query,
                published_date  = r.get("published_date"),
                viral_signals   = detect_viral_signals(content),
            ))
        return posts

    def run(self, topics, platforms, recency="week", max_results=5):
        active = [p for p, on in platforms.items() if on]
        print(f"\n{'='*60}")
        print(f"📡 TAVILY MULTIMODAL AI SCOUT")
        print(f"{'='*60}")
        print(f"   Topics    : {len(topics)} narrow queries")
        print(f"   Platforms : {active}")
        print(f"   Recency   : last {recency}")

        output = ScoutOutput(topics_scouted=topics, platforms_scouted=active)
        seen, q_count = set(), 0

        for topic in topics:
            print(f"\n🔎 '{topic[:60]}'")
            for platform, enabled in platforms.items():
                if not enabled:
                    continue
                for tmpl in QUERY_TEMPLATES.get(platform, [])[:2]:
                    query = tmpl.replace("{topic}", topic)
                    print(f"   [{platform.upper():10s}] {query[:65]}")
                    posts = self._query(query, topic, platform, recency, max_results)
                    q_count += 1
                    new = 0
                    for p in posts:
                        if p.url not in seen:
                            seen.add(p.url)
                            output.posts.append(p)
                            new += 1
                    print(f"              → {new} new | {len(output.posts)} total")
                    time.sleep(0.3)

        output.posts.sort(key=lambda p: (len(p.viral_signals), p.relevance_score), reverse=True)
        output.total_found = len(output.posts)
        output.queries_run = q_count

        print(f"\n{'='*60}")
        print(f"✅ SCOUT COMPLETE — {output.total_found} unique posts")
        for plat, cnt in Counter(p.platform for p in output.posts).most_common():
            print(f"   {plat:15s}: {cnt}")
        print(f"{'='*60}\n")
        return output


# ─────────────────────────────────────────────────────────────────────────────
# YUTORI — Twitter v2 Deep Scout
# ─────────────────────────────────────────────────────────────────────────────

def search_twitter_v2(query, bearer_token, max_results=50):
    """Call Twitter v2 recent search and return engagement-ranked tweets."""
    url     = "https://api.twitter.com/2/tweets/search/recent"
    headers = {"Authorization": f"Bearer {bearer_token}"}
    params  = {
        "query":        query,
        "max_results":  min(max(max_results, 10), 100),
        "tweet.fields": "public_metrics,created_at,author_id,entities",
        "expansions":   "author_id",
        "user.fields":  "name,username,verified,public_metrics",
    }
    try:
        resp = requests.get(url, headers=headers, params=params, timeout=15)
        resp.raise_for_status()
    except Exception as e:
        print(f"   ⚠️  Twitter API error: {e}")
        return []

    data   = resp.json()
    tweets = data.get("data", [])
    users  = {u["id"]: u for u in data.get("includes", {}).get("users", [])}

    results = []
    for t in tweets:
        m      = t.get("public_metrics", {})
        author = users.get(t.get("author_id", ""), {})
        engagement = (
            m.get("like_count", 0)
            + m.get("retweet_count", 0) * 3
            + m.get("reply_count", 0) * 2
            + m.get("quote_count", 0) * 2
            + m.get("impression_count", 0) // 500   # normalize impressions
        )
        results.append({
            "tweet_id":         t["id"],
            "url":              f"https://twitter.com/{author.get('username','')}/status/{t['id']}",
            "text":             t["text"],
            "author":           author.get("name", ""),
            "username":         f"@{author.get('username', '')}",
            "verified":         author.get("verified", False),
            "followers":        author.get("public_metrics", {}).get("followers_count", 0),
            "likes":            m.get("like_count", 0),
            "retweets":         m.get("retweet_count", 0),
            "replies":          m.get("reply_count", 0),
            "quotes":           m.get("quote_count", 0),
            "impressions":      m.get("impression_count", 0),
            "engagement_score": engagement,
            "created_at":       t.get("created_at", ""),
            "query_used":       query,
        })

    results.sort(key=lambda x: x["engagement_score"], reverse=True)
    return results


def run_yutori_multimodal_scout(bearer_token, queries=TWITTER_V2_QUERIES, pull=TWITTER_PULL):
    """Run all narrow Twitter queries, deduplicate, return top N by engagement."""
    print(f"\n{'='*60}")
    print(f"🐦 YUTORI TWITTER DEEP SCOUT — Multimodal AI Focus")
    print(f"{'='*60}")
    print(f"   Queries: {len(queries)}")
    print(f"   Pulling up to {pull} tweets per query")
    print(f"   Geo anchors: {GEO_TERMS}\n")

    seen, all_tweets = set(), []
    for q in queries:
        print(f"🔍 {q[:90]}")
        tweets = search_twitter_v2(q, bearer_token, max_results=pull)
        new = 0
        for t in tweets:
            if t["tweet_id"] not in seen:
                seen.add(t["tweet_id"])
                all_tweets.append(t)
                new += 1
        print(f"   → {new} new | {len(all_tweets)} total")
        time.sleep(0.5)

    all_tweets.sort(key=lambda x: x["engagement_score"], reverse=True)
    print(f"\n✅ {len(all_tweets)} unique tweets collected")
    return all_tweets


# ─────────────────────────────────────────────────────────────────────────────
# DISPLAY
# ─────────────────────────────────────────────────────────────────────────────

PLATFORM_EMOJI = {
    "twitter": "🐦", "youtube": "📹", "reddit": "🤖",
    "linkedin": "💼", "substack": "📝", "medium": "📰",
}
TYPE_EMOJI = {
    "thread": "🧵", "post": "📌", "video": "🎬", "article": "📄", "review": "⭐",
}

def print_top_tweets(tweets, limit=TOP_N_TWEETS):
    print(f"\n{'='*70}")
    print(f"🏆 TOP {limit} VIRAL TWEETS — Multimodal AI Models (Last 7 Days)")
    print(f"   Focus: Models benchmarking at Claude Sonnet / GPT-4o / GPT-5 level")
    print(f"   Geo:   SF · Chicago · New York tech communities")
    print(f"{'='*70}")
    for i, t in enumerate(tweets[:limit]):
        date = t['created_at'][:10] if t['created_at'] else 'N/A'
        print(f"""
#{i+1:02d}  🐦 @{t['username']}  {'✅ verified' if t['verified'] else ''}  ({t['followers']:,} followers)
     📅 {date}
     ❤️  {t['likes']:,}  🔁 {t['retweets']:,}  💬 {t['replies']:,}  🔖 {t['quotes']:,}
     📊 Engagement score: {t['engagement_score']:,}
     🔗 {t['url']}
     💬 {t['text'][:280]}
{'─'*70}""")

def print_tavily_leaderboard(output, limit=10):
    print(f"\n{'='*70}")
    print(f"📡 TAVILY LEADERBOARD — Top {min(limit, output.total_found)} Posts")
    print(f"   (Twitter + YouTube + Reddit | multimodal model releases)")
    print(f"{'='*70}")
    for i, p in enumerate(output.posts[:limit]):
        pe = PLATFORM_EMOJI.get(p.platform, "🌐")
        te = TYPE_EMOJI.get(p.content_type, "📄")
        signals = ", ".join(p.viral_signals[:4]) or "none"
        date_str = f"\n     📅 {p.published_date}" if p.published_date else ""
        print(f"""
#{i+1:02d}  {pe} [{p.platform.upper()}] {te} {p.content_type.upper()}
     📰 {p.title[:75]}
     🔗 {p.url}
     📊 Score: {p.relevance_score}  🔥 {signals}{date_str}
     💬 {p.snippet[:200]}...
{'─'*70}""")


# ─────────────────────────────────────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────────────────────────────────────

if __name__ == "__main__":

    print("🔑 Enter your API credentials")
    TAVILY_API_KEY   = getpass("Tavily API key: ")
    TWITTER_BEARER   = getpass("Twitter Bearer Token (leave blank to skip): ")

    # ── 1. Tavily Scout ───────────────────────────────────────────────────────
    scout        = TavilySocialScout(api_key=TAVILY_API_KEY)
    scout_output = scout.run(
        topics      = TOPICS,
        platforms   = PLATFORMS,
        recency     = RECENCY,
        max_results = MAX_RESULTS,
    )

    # Print Tavily top 10
    print_tavily_leaderboard(scout_output, limit=10)

    # Save Tavily output
    with open("multimodal_tavily_output.json", "w") as f:
        json.dump(asdict(scout_output), f, indent=2, default=str)
    print("\n✅ Saved: multimodal_tavily_output.json")

    # ── 2. Yutori Twitter v2 Deep Scout ──────────────────────────────────────
    if TWITTER_BEARER.strip():
        top_tweets = run_yutori_multimodal_scout(
            bearer_token = TWITTER_BEARER,
            queries      = TWITTER_V2_QUERIES,
            pull         = TWITTER_PULL,
        )

        # ── ★ FINAL OUTPUT: Top 5 Tweets ──────────────────────────────────────
        print_top_tweets(top_tweets, limit=TOP_N_TWEETS)

        # Save Twitter output
        with open("multimodal_twitter_top5.json", "w") as f:
            json.dump(top_tweets[:TOP_N_TWEETS], f, indent=2, default=str)
        print(f"\n✅ Saved: multimodal_twitter_top5.json ({TOP_N_TWEETS} tweets)")

    else:
        print("\nℹ️  Twitter Bearer Token not provided — skipped Yutori.")
        print("   Add your token above to enable Twitter v2 deep search.")

    print("\n🏁 Done.")

🔑 Enter your API credentials
Tavily API key: ··········
Twitter Bearer Token (leave blank to skip): ··········

📡 TAVILY MULTIMODAL AI SCOUT
   Topics    : 5 narrow queries
   Platforms : ['twitter', 'youtube', 'reddit']
   Recency   : last week

🔎 'multimodal model benchmark Claude Sonnet GPT-4o'
   [TWITTER   ] multimodal model benchmark Claude Sonnet GPT-4o site:twitter.com 
              → 5 new | 5 total
   [TWITTER   ] multimodal model benchmark Claude Sonnet GPT-4o site:x.com releas
              → 3 new | 8 total
   [YOUTUBE   ] multimodal model benchmark Claude Sonnet GPT-4o site:youtube.com 
              → 5 new | 13 total
   [YOUTUBE   ] multimodal model benchmark Claude Sonnet GPT-4o youtube multimoda
              → 5 new | 18 total
   [REDDIT    ] multimodal model benchmark Claude Sonnet GPT-4o site:reddit.com r
              → 5 new | 23 total
   [REDDIT    ] multimodal model benchmark Claude Sonnet GPT-4o reddit multimodal
              → 5 new | 28 total

🔎 'new visio